In [27]:
import sys
import beessna.bib.preprocessing as prep
import pandas as pd
import numpy as np
import networkx as nx
from scipy import spatial

from bb_binary import FrameContainer, Repository, load_frame_container
from pandas import DataFrame, Series

%load_ext line_profiler

In [3]:
def get_files(path):
    repo = Repository(path)
    file = list(repo.iter_fnames())
    a = [f.split('/')[-1].split("_")[1] for f in file]
    l = len(a)/4
    npa = np.array(file).reshape(int(l),4)
    return npa

In [4]:
def get_close_bees(df, distance):

    df = df.reset_index(level = 'frame_idx')

    m = pd.merge(df, df, on='frame_idx')
    #m = m.query('id_x < id_y')
    m = m[m.id_x < m.id_y]

    m.loc[:, 'dist'] = np.square(m.xpos_x - m.xpos_y) \
        + np.square(m.ypos_x - m.ypos_y)

    filtered = m[m.dist <= distance**2]

    filtered = filtered[['frame_idx','id_x', 'id_y']]
    return filtered

In [5]:
def get_close_bees_kd(df, distance):

    df_close = DataFrame()

    gr = df.groupby('frame_idx')

    for i, group in gr:
        xy_coordinates = group[['xpos', 'ypos']].values
        tree = spatial.KDTree(xy_coordinates, leafsize=20)
        result = tree.query_pairs(distance)
        l = [[i,group['id'].iat[a], group['id'].iat[b]] for a,b in result]
        df_close = df_close.append(DataFrame(l, columns=['frame_idx', 'id_x', 'id_y']))

    return df_close

In [20]:
def get_close_bees_ckd(df, distance):

    df_close = DataFrame()

    gr = df.groupby('frame_idx')

    for i, group in gr:
        xy_coordinates = group[['xpos', 'ypos']].values
        tree = spatial.cKDTree(xy_coordinates, leafsize=20)
        result = tree.query_pairs(distance)
        l = [[i,group['id'].iat[a], group['id'].iat[b]] for a,b in result]
        df_close = df_close.append(DataFrame(l, columns=['frame_idx', 'id_x', 'id_y']))

    return df_close

In [6]:
global side1
global side2

In [32]:
def f(c=0.95,d=160,m="10",l=3):
    
    f = "data/1h/"
    p = "2015" + m + "2215"

    CONFIDENCE = c
    DISTANCE = d

    xmax = 3000
    ymax = 4000
    LENGTH = l

    path = f+p

    files = get_files(path)

    interactions = Series()

    for file_list in files[:1]:

        dataframes = np.empty(4, dtype=object)

        for i in list(range(4)):
            fc = load_frame_container(file_list[i])
            df = prep.get_dataframe(fc)
            df = prep.calcIds(df,CONFIDENCE)

            camIdx = int(file_list[i].split("/")[-1].split("_")[1])
            dataframes[camIdx] = df

        # cam 0 und cam1 nach rechts verschieben
        dataframes[0].xpos = dataframes[0].xpos + xmax
        dataframes[1].xpos = dataframes[1].xpos + xmax

        # Seiten zusammenfugen
        side0 = pd.concat([dataframes[3], dataframes[0]])
        side1 = pd.concat([dataframes[2], dataframes[1]])

        close1 = get_close_bees_ckd(side0, DISTANCE)
        close2 = get_close_bees_ckd(side1, DISTANCE)

        close = pd.concat([close1,close2])

        p = prep.bee_pairs_to_timeseries(close)

        edges = prep.extract_interactions(p,LENGTH)

        interactions = pd.concat([interactions, edges])

    G = prep.create_graph2(interactions)
    # nx.write_graphml(G, filename + ".graphml")

In [33]:
%lprun -f f f(m="08")

In [34]:
%time f(m="08")

CPU times: user 1min 31s, sys: 300 ms, total: 1min 31s
Wall time: 1min 31s


# August

Timer unit: 1e-06 s

Total time: 289.339 s
File: <ipython-input-28-15d62d1629c7>
Function: f at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def f(c=0.95,d=160,m="10",l=3):
     2                                               
     3         1            4      4.0      0.0      f = "data/1h/"
     4         1            5      5.0      0.0      p = "2015" + m + "2215"
     5                                           
     6         1            2      2.0      0.0      CONFIDENCE = c
     7         1            2      2.0      0.0      DISTANCE = d
     8                                           
     9         1            2      2.0      0.0      xmax = 3000
    10         1            2      2.0      0.0      ymax = 4000
    11         1            2      2.0      0.0      LENGTH = l
    12                                           
    13         1            3      3.0      0.0      path = f+p
    14                                           
    15         1        36183  36183.0      0.0      files = get_files(path)
    16                                           
    17         1          794    794.0      0.0      interactions = Series()
    18                                           
    19         2           17      8.5      0.0      for file_list in files[:1]:
    20                                           
    21         1           17     17.0      0.0          dataframes = np.empty(4, dtype=object)
    22                                           
    23         5           20      4.0      0.0          for i in list(range(4)):
    24         4        18684   4671.0      0.0              fc = load_frame_container(file_list[i])
    25         4     34087673 8521918.2     11.8              df = prep.get_dataframe(fc)
    26         4     31822032 7955508.0     11.0              df = prep.calcIds(df,CONFIDENCE)
    27                                           
    28         4          295     73.8      0.0              camIdx = int(file_list[i].split("/")[-1].split("_")[1])
    29         4           14      3.5      0.0              dataframes[camIdx] = df
    30                                           
    31                                                   # cam 0 und cam1 nach rechts verschieben
    32         1         2147   2147.0      0.0          dataframes[0].xpos = dataframes[0].xpos + xmax
    33         1         1201   1201.0      0.0          dataframes[1].xpos = dataframes[1].xpos + xmax
    34                                           
    35                                                   # Seiten zusammenfugen
    36         1        17167  17167.0      0.0          side0 = pd.concat([dataframes[3], dataframes[0]])
    37         1        11736  11736.0      0.0          side1 = pd.concat([dataframes[2], dataframes[1]])
    38                                           
    39         1     16361026 16361026.0      5.7          close1 = get_close_bees_ckd(side0, DISTANCE)
    40         1      8482647 8482647.0      2.9          close2 = get_close_bees_ckd(side1, DISTANCE)
    41                                           
    42         1     90526887 90526887.0     31.3          close1 = get_close_bees_kd(side0, DISTANCE)
    43         1     38827023 38827023.0     13.4          close2 = get_close_bees_kd(side1, DISTANCE)
    44                                           
    45         1     29052971 29052971.0     10.0          close1 = get_close_bees(side0, DISTANCE)
    46         1      6008248 6008248.0      2.1          close2 = get_close_bees(side1, DISTANCE)
    47                                           
    48         1         2511   2511.0      0.0          close = pd.concat([close1,close2])
    49                                           
    50         1     11966027 11966027.0      4.1          p = prep.bee_pairs_to_timeseries(close)
    51                                           
    52         1     22061077 22061077.0      7.6          edges = prep.extract_interactions(p,LENGTH)
    53                                           
    54         1         2143   2143.0      0.0          interactions = pd.concat([interactions, edges])
    55                                           
    56         1        50235  50235.0      0.0      G = prep.create_graph2(interactions)

# September

Timer unit: 1e-06 s

Total time: 112.756 s
File: <ipython-input-28-15d62d1629c7>
Function: f at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def f(c=0.95,d=160,m="10",l=3):
     2                                               
     3         1            4      4.0      0.0      f = "data/1h/"
     4         1            4      4.0      0.0      p = "2015" + m + "2215"
     5                                           
     6         1            2      2.0      0.0      CONFIDENCE = c
     7         1            2      2.0      0.0      DISTANCE = d
     8                                           
     9         1            2      2.0      0.0      xmax = 3000
    10         1            2      2.0      0.0      ymax = 4000
    11         1            2      2.0      0.0      LENGTH = l
    12                                           
    13         1            2      2.0      0.0      path = f+p
    14                                           
    15         1        41787  41787.0      0.0      files = get_files(path)
    16                                           
    17         1          812    812.0      0.0      interactions = Series()
    18                                           
    19         2           16      8.0      0.0      for file_list in files[:1]:
    20                                           
    21         1           17     17.0      0.0          dataframes = np.empty(4, dtype=object)
    22                                           
    23         5           20      4.0      0.0          for i in list(range(4)):
    24         4         7953   1988.2      0.0              fc = load_frame_container(file_list[i])
    25         4     27026549 6756637.2     24.0              df = prep.get_dataframe(fc)
    26         4     12670936 3167734.0     11.2              df = prep.calcIds(df,CONFIDENCE)
    27                                           
    28         4          286     71.5      0.0              camIdx = int(file_list[i].split("/")[-1].split("_")[1])
    29         4           14      3.5      0.0              dataframes[camIdx] = df
    30                                           
    31                                                   # cam 0 und cam1 nach rechts verschieben
    32         1         1573   1573.0      0.0          dataframes[0].xpos = dataframes[0].xpos + xmax
    33         1         1107   1107.0      0.0          dataframes[1].xpos = dataframes[1].xpos + xmax
    34                                           
    35                                                   # Seiten zusammenfugen
    36         1        10340  10340.0      0.0          side0 = pd.concat([dataframes[3], dataframes[0]])
    37         1         9029   9029.0      0.0          side1 = pd.concat([dataframes[2], dataframes[1]])
    38                                           
    39         1      7494383 7494383.0      6.6          close1 = get_close_bees_ckd(side0, DISTANCE)
    40         1      6088745 6088745.0      5.4          close2 = get_close_bees_ckd(side1, DISTANCE)
    41                                           
    42         1     29403785 29403785.0     26.1          close1 = get_close_bees_kd(side0, DISTANCE)
    43         1     15735002 15735002.0     14.0          close2 = get_close_bees_kd(side1, DISTANCE)
    44                                           
    45         1      3877745 3877745.0      3.4          close1 = get_close_bees(side0, DISTANCE)
    46         1      1407698 1407698.0      1.2          close2 = get_close_bees(side1, DISTANCE)
    47                                           
    48         1         1610   1610.0      0.0          close = pd.concat([close1,close2])
    49                                           
    50         1      4264701 4264701.0      3.8          p = prep.bee_pairs_to_timeseries(close)
    51                                           
    52         1      4695635 4695635.0      4.2          edges = prep.extract_interactions(p,LENGTH)
    53                                           
    54         1         1963   1963.0      0.0          interactions = pd.concat([interactions, edges])
    55                                           
    56         1        14282  14282.0      0.0      G = prep.create_graph2(interactions)


# Oktober

Timer unit: 1e-06 s

Total time: 51.0593 s
File: <ipython-input-28-15d62d1629c7>
Function: f at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def f(c=0.95,d=160,m="10",l=3):
     2                                               
     3         1            4      4.0      0.0      f = "data/1h/"
     4         1            5      5.0      0.0      p = "2015" + m + "2215"
     5                                           
     6         1            2      2.0      0.0      CONFIDENCE = c
     7         1            2      2.0      0.0      DISTANCE = d
     8                                           
     9         1            2      2.0      0.0      xmax = 3000
    10         1            2      2.0      0.0      ymax = 4000
    11         1            2      2.0      0.0      LENGTH = l
    12                                           
    13         1            2      2.0      0.0      path = f+p
    14                                           
    15         1        40525  40525.0      0.1      files = get_files(path)
    16                                           
    17         1          756    756.0      0.0      interactions = Series()
    18                                           
    19         2           16      8.0      0.0      for file_list in files[:1]:
    20                                           
    21         1           18     18.0      0.0          dataframes = np.empty(4, dtype=object)
    22                                           
    23         5           17      3.4      0.0          for i in list(range(4)):
    24         4         4166   1041.5      0.0              fc = load_frame_container(file_list[i])
    25         4     23563753 5890938.2     46.1              df = prep.get_dataframe(fc)
    26         4      2648676 662169.0      5.2              df = prep.calcIds(df,CONFIDENCE)
    27                                           
    28         4          113     28.2      0.0              camIdx = int(file_list[i].split("/")[-1].split("_")[1])
    29         4           10      2.5      0.0              dataframes[camIdx] = df
    30                                           
    31                                                   # cam 0 und cam1 nach rechts verschieben
    32         1         1197   1197.0      0.0          dataframes[0].xpos = dataframes[0].xpos + xmax
    33         1         1023   1023.0      0.0          dataframes[1].xpos = dataframes[1].xpos + xmax
    34                                           
    35                                                   # Seiten zusammenfugen
    36         1         7299   7299.0      0.0          side0 = pd.concat([dataframes[3], dataframes[0]])
    37         1         7155   7155.0      0.0          side1 = pd.concat([dataframes[2], dataframes[1]])
    38                                           
    39         1      5143742 5143742.0     10.1          close1 = get_close_bees_ckd(side0, DISTANCE)
    40         1      5120215 5120215.0     10.0          close2 = get_close_bees_ckd(side1, DISTANCE)
    41                                           
    42         1      6173559 6173559.0     12.1          close1 = get_close_bees_kd(side0, DISTANCE)
    43         1      6143488 6143488.0     12.0          close2 = get_close_bees_kd(side1, DISTANCE)
    44                                           
    45         1       123014 123014.0      0.2          close1 = get_close_bees(side0, DISTANCE)
    46         1       125967 125967.0      0.2          close2 = get_close_bees(side1, DISTANCE)
    47                                           
    48         1         1341   1341.0      0.0          close = pd.concat([close1,close2])
    49                                           
    50         1      1808195 1808195.0      3.5          p = prep.bee_pairs_to_timeseries(close)
    51                                           
    52         1       142805 142805.0      0.3          edges = prep.extract_interactions(p,LENGTH)
    53                                           
    54         1         1706   1706.0      0.0          interactions = pd.concat([interactions, edges])
    55                                           
    56         1          503    503.0      0.0      G = prep.create_graph2(interactions)

Timer unit: 1e-06 s

Total time: 131.67 s
File: <ipython-input-32-2edbd3501b0b>
Function: f at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def f(c=0.95,d=160,m="10",l=3):
     2                                               
     3         1            3      3.0      0.0      f = "data/1h/"
     4         1            3      3.0      0.0      p = "2015" + m + "2215"
     5                                           
     6         1            2      2.0      0.0      CONFIDENCE = c
     7         1            1      1.0      0.0      DISTANCE = d
     8                                           
     9         1            1      1.0      0.0      xmax = 3000
    10         1            2      2.0      0.0      ymax = 4000
    11         1            2      2.0      0.0      LENGTH = l
    12                                           
    13         1            2      2.0      0.0      path = f+p
    14                                           
    15         1        27978  27978.0      0.0      files = get_files(path)
    16                                           
    17         1          584    584.0      0.0      interactions = Series()
    18                                           
    19         2           15      7.5      0.0      for file_list in files[:1]:
    20                                           
    21         1           13     13.0      0.0          dataframes = np.empty(4, dtype=object)
    22                                           
    23         5           16      3.2      0.0          for i in list(range(4)):
    24         4        17682   4420.5      0.0              fc = load_frame_container(file_list[i])
    25         4     34144405 8536101.2     25.9              df = prep.get_dataframe(fc)
    26         4     33137433 8284358.2     25.2              df = prep.calcIds(df,CONFIDENCE)
    27                                           
    28         4          202     50.5      0.0              camIdx = int(file_list[i].split("/")[-1].split("_")[1])
    29         4           15      3.8      0.0              dataframes[camIdx] = df
    30                                           
    31                                                   # cam 0 und cam1 nach rechts verschieben
    32         1         2113   2113.0      0.0          dataframes[0].xpos = dataframes[0].xpos + xmax
    33         1         1199   1199.0      0.0          dataframes[1].xpos = dataframes[1].xpos + xmax
    34                                           
    35                                                   # Seiten zusammenfugen
    36         1        17273  17273.0      0.0          side0 = pd.concat([dataframes[3], dataframes[0]])
    37         1        11689  11689.0      0.0          side1 = pd.concat([dataframes[2], dataframes[1]])
    38                                           
    39         1     15639772 15639772.0     11.9          close1 = get_close_bees_ckd(side0, DISTANCE)
    40         1      8425304 8425304.0      6.4          close2 = get_close_bees_ckd(side1, DISTANCE)
    41                                           
    42         1         3158   3158.0      0.0          close = pd.concat([close1,close2])
    43                                           
    44         1     11007595 11007595.0      8.4          p = prep.bee_pairs_to_timeseries(close)
    45                                           
    46         1     29174109 29174109.0     22.2          edges = prep.extract_interactions(p,LENGTH)
    47                                           
    48         1         2219   2219.0      0.0          interactions = pd.concat([interactions, edges])
    49                                           
    50         1        56727  56727.0      0.0      G = prep.create_graph2(interactions)

# Optimizing calc IDs!

In [52]:
def f2(c=0.95,d=160,m="10",l=3):
    
    f = "data/1h/"
    p = "2015" + m + "2215"

    CONFIDENCE = c
    DISTANCE = d

    xmax = 3000
    ymax = 4000
    LENGTH = l

    path = f+p

    files = get_files(path)

    for file_list in files[:1]:

        for i in list(range(4)):
            fc = load_frame_container(file_list[i])
            df = prep.get_dataframe(fc)
            df = calcIds(df,CONFIDENCE)

In [104]:
# helper function
# Zum ausrechnen der IDs
def get_detected_id(bits):
    binary_id = (bits>0.5)*1
    decimal_id = int(''.join([str(c) for c in binary_id.tolist()[:11]]), 2)

    # determine what kind of parity bit was used and add 2^11 to decimal id
    # uneven parity bit was used
    if ((sum(binary_id) % 2) == 1):
        decimal_id += 2048

    return decimal_id

In [103]:
def get_confidence(bits):
    return np.min(np.abs(0.5 - bits)) * 2

In [119]:
# Dezimale ID ausrechnen und an DataFrame angaengen
def calcIds(df, threshold):
    df.decodedId = df.decodedId.apply(lambda x: np.array(x)/255)
    df = df.assign(confidence = df.decodedId.apply(get_confidence))
    df = df[df.confidence >= threshold]
    df = df.assign(id = df.decodedId.apply(get_detected_id))

    df = df.drop('decodedId', 1)

    return df

In [108]:
%lprun -f f2 -f calcIds -f get_confidence -f get_detected_id f2(m="08")

*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-06 s

Total time: 74.6098 s
File: <ipython-input-52-081516ed82b8>
Function: f2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def f2(c=0.95,d=160,m="10",l=3):
     2                                               
     3         1            2      2.0      0.0      f = "data/1h/"
     4         1            2      2.0      0.0      p = "2015" + m + "2215"
     5                                           
     6         1            1      1.0      0.0      CONFIDENCE = c
     7         1            0      0.0      0.0      DISTANCE = d
     8                                           
     9         1            0      0.0      0.0      xmax = 3000
    10         1            1      1.0      0.0      ymax = 4000
    11         1            1      1.0      0.0      LENGTH = l
    12                                           
    13         1            1      1.0      0.0      path = f+p
    14                                           
    15         1        28104  28104.0      0.0      files = get_files(path)
    16                                           
    17         2           27     13.5      0.0      for file_list in files[:1]:
    18                                           
    19         5           29      5.8      0.0          for i in list(range(4)):
    20         4        16397   4099.2      0.0              fc = load_frame_container(file_list[i])
    21         4     34193878 8548469.5     45.8              df = prep.get_dataframe(fc)
    22         4     40371374 10092843.5     54.1              df = calcIds(df,CONFIDENCE)

Total time: 16.6785 s
File: <ipython-input-53-72087af56ba1>
Function: get_detected_id at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     3                                           def get_detected_id(bits):
     4                                           
     5                                               # Umrechnen in binary array [0,1,1,1,0,1,1,1,0,0,0,1]
     6    319929      2024772      6.3     12.1      bits = np.array(bits)
     7    319929      1896065      5.9     11.4      bits = bits/255
     8    319929      7356271     23.0     44.1      binary_id = [int(x > 0.5) for x in bits]
     9                                           
    10    319929      3841928     12.0     23.0      decimal_id = int(''.join([str(c) for c in binary_id[:11]]), 2)
    11                                           
    12                                               # determine what kind of parity bit was used and add 2^11 to decimal id
    13                                               # uneven parity bit was used
    14    319929       824578      2.6      4.9      if ((sum(binary_id) % 2) == 1):
    15    264078       367062      1.4      2.2          decimal_id += 2048
    16                                           
    17    319929       367813      1.1      2.2      return decimal_id

Total time: 17.2008 s
File: <ipython-input-54-2261ddafb2f4>
Function: get_confidence at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_confidence(bits):
     2    504847      3442396      6.8     20.0      bits = np.array(bits)
     3    504847      2841899      5.6     16.5      bits = bits/255
     4                                           
     5    504847     10916481     21.6     63.5      return np.min(np.abs(0.5 - bits)) * 2

Total time: 40.2917 s
File: <ipython-input-55-cf48b804dfce>
Function: calcIds at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           def calcIds(df, threshold):
     3         4     19315084 4828771.0     47.9      df = df.assign(confidence = df.decodedId.apply(get_confidence))
     4         4        50012  12503.0      0.1      df = df[df.confidence >= threshold]
     5         4     20891742 5222935.5     51.9      df = df.assign(id = df.decodedId.apply(get_detected_id))
     6                                           
     7         4        34879   8719.8      0.1      df = df.drop('decodedId', 1)
     8                                           
     9         4           17      4.2      0.0      return df

In [ ]:
bits = np.random.rand(1000000)

In [66]:
%timeit binary_id = [int(x > 0.5) for x in bits]

1 loop, best of 3: 856 ms per loop


In [72]:
%timeit (bits>0.5)*1

100 loops, best of 3: 4.06 ms per loop


In [76]:
%lprun -f f2 -f calcIds -f get_confidence -f get_detected_id f2(m="08")

In [82]:
%timeit binary_id = [int(x > 0.5) for x in bits]

1 loop, best of 3: 878 ms per loop


In [83]:
%timeit binary_id2 = (bits>0.5)*1

100 loops, best of 3: 3.97 ms per loop


In [95]:
%timeit decimal_id = int(''.join([str(c) for c in binary_id]), 2)

1 loop, best of 3: 349 ms per loop


In [94]:
%timeit decimal_id = int(''.join([str(c) for c in binary_id2.tolist()]), 2)

1 loop, best of 3: 360 ms per loop


In [98]:
%lprun -f f2 -f calcIds -f get_confidence -f get_detected_id f2(m="08")

In [101]:
sum(binary_id)

499803

Timer unit: 1e-06 s

Total time: 72.5807 s
File: <ipython-input-52-081516ed82b8>
Function: f2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def f2(c=0.95,d=160,m="10",l=3):
     2                                               
     3         1            3      3.0      0.0      f = "data/1h/"
     4         1            3      3.0      0.0      p = "2015" + m + "2215"
     5                                           
     6         1            1      1.0      0.0      CONFIDENCE = c
     7         1            1      1.0      0.0      DISTANCE = d
     8                                           
     9         1            1      1.0      0.0      xmax = 3000
    10         1            1      1.0      0.0      ymax = 4000
    11         1            1      1.0      0.0      LENGTH = l
    12                                           
    13         1            2      2.0      0.0      path = f+p
    14                                           
    15         1        24711  24711.0      0.0      files = get_files(path)
    16                                           
    17         2           27     13.5      0.0      for file_list in files[:1]:
    18                                           
    19         5           29      5.8      0.0          for i in list(range(4)):
    20         4        19239   4809.8      0.0              fc = load_frame_container(file_list[i])
    21         4     34149115 8537278.8     47.0              df = prep.get_dataframe(fc)
    22         4     38387586 9596896.5     52.9              df = calcIds(df,CONFIDENCE)

Total time: 17.4818 s
File: <ipython-input-54-2261ddafb2f4>
Function: get_confidence at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_confidence(bits):
     2    504847      3523779      7.0     20.2      bits = np.array(bits)
     3    504847      2874457      5.7     16.4      bits = bits/255
     4                                           
     5    504847     11083518     22.0     63.4      return np.min(np.abs(0.5 - bits)) * 2

Total time: 38.308 s
File: <ipython-input-55-cf48b804dfce>
Function: calcIds at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           def calcIds(df, threshold):
     3         4     19629457 4907364.2     51.2      df = df.assign(confidence = df.decodedId.apply(get_confidence))
     4         4        52273  13068.2      0.1      df = df[df.confidence >= threshold]
     5         4     18588510 4647127.5     48.5      df = df.assign(id = df.decodedId.apply(get_detected_id))
     6                                           
     7         4        37755   9438.8      0.1      df = df.drop('decodedId', 1)
     8                                           
     9         4           17      4.2      0.0      return df

Total time: 15.0665 s
File: <ipython-input-97-61d3f5fa6144>
Function: get_detected_id at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     3                                           def get_detected_id(bits):
     4                                           
     5                                               # Umrechnen in binary array [0,1,1,1,0,1,1,1,0,0,0,1]
     6    319929      2050586      6.4     13.6      bits = np.array(bits)
     7    319929      1933509      6.0     12.8      bits = bits/255
     8    319929      2811215      8.8     18.7      binary_id = (bits>0.5)*1
     9                                           
    10    319929      4760327     14.9     31.6      decimal_id = int(''.join([str(c) for c in binary_id.tolist()[:11]]), 2)
    11                                           
    12                                               # determine what kind of parity bit was used and add 2^11 to decimal id
    13                                               # uneven parity bit was used
    14    319929      2849710      8.9     18.9      if ((sum(binary_id) % 2) == 1):
    15    264078       350912      1.3      2.3          decimal_id += 2048
    16                                           
    17    319929       310214      1.0      2.1      return decimal_id

In [120]:
%lprun -f f2 -f calcIds -f get_confidence -f get_detected_id f2(m="08")

Timer unit: 1e-06 s

Total time: 10.4425 s
File: <ipython-input-103-45d845ac9a0b>
Function: get_confidence at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_confidence(bits):
     2    504847     10442488     20.7    100.0      return np.min(np.abs(0.5 - bits)) * 2

Total time: 10.8666 s
File: <ipython-input-104-a0bbeffc4959>
Function: get_detected_id at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     3                                           def get_detected_id(bits):
     4    319929      3131722      9.8     28.8      binary_id = (bits>0.5)*1
     5    319929      4490893     14.0     41.3      decimal_id = int(''.join([str(c) for c in binary_id.tolist()[:11]]), 2)
     6                                           
     7                                               # determine what kind of parity bit was used and add 2^11 to decimal id
     8                                               # uneven parity bit was used
     9    319929      2629997      8.2     24.2      if ((sum(binary_id) % 2) == 1):
    10    264078       324526      1.2      3.0          decimal_id += 2048
    11                                           
    12    319929       289433      0.9      2.7      return decimal_id

Total time: 29.5378 s
File: <ipython-input-119-d82846765835>
Function: calcIds at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           def calcIds(df, threshold):
     3         4      4278492 1069623.0     14.5      df.decodedId = df.decodedId.apply(lambda x: np.array(x)/255)
     4         4     11776394 2944098.5     39.9      df = df.assign(confidence = df.decodedId.apply(get_confidence))
     5         4        55361  13840.2      0.2      df = df[df.confidence >= threshold]
     6         4     13391031 3347757.8     45.3      df = df.assign(id = df.decodedId.apply(get_detected_id))
     7                                           
     8         4        36533   9133.2      0.1      df = df.drop('decodedId', 1)
     9                                           
    10         4           17      4.2      0.0      return df

Total time: 64.3705 s
File: <ipython-input-52-081516ed82b8>
Function: f2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def f2(c=0.95,d=160,m="10",l=3):
     2                                               
     3         1            3      3.0      0.0      f = "data/1h/"
     4         1            4      4.0      0.0      p = "2015" + m + "2215"
     5                                           
     6         1            1      1.0      0.0      CONFIDENCE = c
     7         1            1      1.0      0.0      DISTANCE = d
     8                                           
     9         1            1      1.0      0.0      xmax = 3000
    10         1            0      0.0      0.0      ymax = 4000
    11         1            1      1.0      0.0      LENGTH = l
    12                                           
    13         1            1      1.0      0.0      path = f+p
    14                                           
    15         1        27978  27978.0      0.0      files = get_files(path)
    16                                           
    17         2           26     13.0      0.0      for file_list in files[:1]:
    18                                           
    19         5           26      5.2      0.0          for i in list(range(4)):
    20         4        20161   5040.2      0.0              fc = load_frame_container(file_list[i])
    21         4     34601416 8650354.0     53.8              df = prep.get_dataframe(fc)
    22         4     29720903 7430225.8     46.2              df = calcIds(df,CONFIDENCE)

In [127]:
def f3(c=0.95,d=160,m="10",l=3):
    
    f = "data/1h/"
    p = "2015" + m + "2215"

    CONFIDENCE = c
    DISTANCE = d

    xmax = 3000
    ymax = 4000
    LENGTH = l

    path = f+p

    files = get_files(path)

    interactions = Series()

    for file_list in files[:1]:

        dataframes = np.empty(4, dtype=object)

        for i in list(range(4)):
            fc = load_frame_container(file_list[i])
            df = prep.get_dataframe(fc)
            df = prep.calcIds(df,CONFIDENCE)

            camIdx = int(file_list[i].split("/")[-1].split("_")[1])
            dataframes[camIdx] = df

        # cam 0 und cam1 nach rechts verschieben
        dataframes[0].xpos = dataframes[0].xpos + xmax
        dataframes[1].xpos = dataframes[1].xpos + xmax

        # Seiten zusammenfugen
        side0 = pd.concat([dataframes[3], dataframes[0]])
        side1 = pd.concat([dataframes[2], dataframes[1]])

        close1 = get_close_bees_ckd(side0, DISTANCE)
        close2 = get_close_bees_ckd(side1, DISTANCE)

        close = pd.concat([close1,close2])

        p = prep.bee_pairs_to_timeseries(close)

        edges = extract_interactions(p,LENGTH)

        interactions = pd.concat([interactions, edges])

    G = prep.create_graph2(interactions)
    # nx.write_graphml(G, filename + ".graphml")

In [128]:
def extract_interactions(dft, minlength):
    kette = dft.apply(get_ketten, axis=1, args=["0"])
    kk = kette.apply(lambda x: [len(item) for item in x])
    kk = kk.apply(lambda x: len([item for item in x if item >= minlength]))
    return kk[kk > 0]

In [129]:
def get_ketten(kette, val):
    kette = kette.apply(str)
    s = kette.str.cat(sep='')
    ss = s.split(val)
    return [x for x in ss if len(x) > 0]

In [146]:
%time f3(m="08")

CPU times: user 1min 31s, sys: 232 ms, total: 1min 32s
Wall time: 1min 31s


In [141]:
s = Series(np.array([1]*1000000))

In [143]:
%timeit s.apply(str)

1 loop, best of 3: 333 ms per loop


In [145]:
%timeit ''.join([str(c) for c in s.tolist()])

1 loop, best of 3: 399 ms per loop
